In [1]:
import keras
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
plt.style.use('dark_background')

import os
def plot_imgs(directory, top=10):
    all_item_dirs = os.listdir(directory)
    item_files = [os.path.join(directory, file) for file in all_item_dirs][:5]

    plt.figure(figsize=(20, 20))

    for i, img_path in enumerate(item_files):
        plt.subplot(10, 10, i+1)

        img = plt.imread(img_path)
        plt.tight_layout()
        plt.imshow(img, cmap='gray')

data_path = './eyes/dataset/train'

directories = ['/Closed_Eyes', '/Open_Eyes']

for j in directories:
    plot_imgs(data_path+j)

batch_size = 32
train_datagen = ImageDataGenerator(horizontal_flip = True,
                                  rescale = 1./255,
                                  zoom_range = 0.2,
                                  validation_split = 0.1)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_data_path = './eyes/dataset/train'
test_data_path = './eyes/dataset/test'

train_set = train_datagen.flow_from_directory(train_data_path, target_size = (64,64),
                                              batch_size = batch_size,
                                              color_mode = 'grayscale',
                                              class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_data_path, target_size = (64,64),
                                              batch_size = batch_size,
                                              color_mode = 'grayscale',
                                              class_mode = 'categorical')

classes = 2

model = Sequential()
model.add(Conv2D(32, (3,3), padding = 'same', input_shape = (64,64,1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64, (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,(3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))

model.add(Dense(classes, activation = 'softmax'))

print(model.summary())

model.compile(loss = 'categorical_crossentropy',optimizer = 'adam' , metrics = ['accuracy'])

model_path="yawn_detection.h5"

checkpoint = ModelCheckpoint(model_path, monitor='val_accuracy', verbose=1,
                              save_best_only=True, mode='max')

callbacks_list = [checkpoint]

num_epochs = 20
training_steps=train_set.n//train_set.batch_size
validation_steps =test_set.n//test_set.batch_size

history = model.fit_generator(train_set, epochs=num_epochs, steps_per_epoch=training_steps,validation_data=test_set,
                    validation_steps=validation_steps, callbacks = callbacks_list)


Found 3400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 32, 32, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                    

C:\Users\이소정\AppData\Local\Temp\ipykernel_556\360112266.py:84: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_set, epochs=num_epochs, steps_per_epoch=training_steps,validation_data=test_set,


Epoch 1/20


106/106 [==============================] - ETA: 0s - loss: 0.2653 - accuracy: 0.8806
Epoch 1: val_accuracy improved from -inf to 0.99479, saving model to yawn_detection.h5
106/106 [==============================] - 25s 210ms/step - loss: 0.2653 - accuracy: 0.8806 - val_loss: 0.0101 - val_accuracy: 0.9948
Epoch 2/20


C:\Users\이소정\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


106/106 [==============================] - ETA: 0s - loss: 0.0749 - accuracy: 0.9757
Epoch 2: val_accuracy improved from 0.99479 to 1.00000, saving model to yawn_detection.h5
106/106 [==============================] - 17s 163ms/step - loss: 0.0749 - accuracy: 0.9757 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 3/20
 63/106 [================>.............] - ETA: 7s - loss: 0.0323 - accuracy: 0.9890

In [ ]:
from keras.preprocessing import image
import numpy as np

# 이미지 경로
image_path = './closed.PNG'

# 이미지 불러오기
img = image.load_img(image_path, target_size=(64, 64), color_mode='grayscale')

# 이미지를 모델에 입력 가능한 형태로 변환
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # 모델 훈련 시 rescale 값과 동일하게 이미지 정규화

# 모델 예측
prediction = model.predict(img_array)

# 예측 결과 출력
if prediction[0][0] > prediction[0][1]:
    print("눈을 감은 상태입니다.")
else:
    print("눈을 떴습니다.")


1/1 [==============================] - 0s 325ms/step
눈을 감은 상태입니다.
